# Теоремы об исключении и продолжении

## Упражнения к параграфу 1

### Упражнение 2

Рассмотрим систему уравнений $$x^2 + 2y^2 = 3$$
                         $$x^2 + xy + y^2 = 3$$
(a) Пусть $I = (x^2 + 2y^2 - 3, x^2 + xy + y^2 - 3)$ - соответствующий идеал.
Найти базисы идеалов $I\cap k[x]$ и $I\cap k[y]$.

(b) Найти все решения системы.

(c) Определить, какие из этих решений рациональны.

(d) Найти наименьшее поле $k$ такое, что все решения принадлежат $k^2$.

**Решение.**

По **теореме об исключении** базис $G_1$ идеала $I_1 = I\cap k[x]$ равен $G \cap k[x]$, где $G$ - базис Грёбнера идеала $I$.

Тогда для начала найдём базис Грёбнера $G$ относительно лексикографического порядка $y > x$. Для этого нам понадобится функция $groebner$ из библиотеки $SymPy$.

In [19]:
from sympy.polys.polytools import groebner
from sympy.abc import x, y

F = [y**2 + x*y + x**2 - 3, 2*y**2 + x**2 - 3]           # Идеал I
g = groebner(F, y, x, order='lex')                       # Базис Грёбнера g

print('\nБазис Грёбнера относительно y > x:')
for i in range(len(g.exprs)):
    print('g', i+1, ' = ', g.exprs[i], sep='')           # метод .exprs позволяет записать полином в виде красивого выражения


Базис Грёбнера относительно y > x:
g1 = x**3 - 3*x + 2*y
g2 = x**4 - 4*x**2 + 3


Теперь несложно найти $G_1 = G \cap k[x]$:

In [21]:
from sympy import Poly, LC
from sympy import solve, solve_poly_system

polys = g.exprs
G1 = []
index = 1
for pol in polys:
    f = Poly(pol, x)                                     # Представим каждое выражение как полином от переменной X
    if f.free_symbols_in_domain != {y}:                  # Если помимо переменной X в полиноме НЕТ свободной переменной Y,
        G1.append((f.as_expr(), index))                  # то этот полином содержится в G1
    index += 1
print('\nБазис Грёбнера 1-ого исключающего идеала:')
for pol in G1:
    print('g', pol[1], ' = ', pol[0], sep='')
print('\n')


Базис Грёбнера 1-ого исключающего идеала:
g2 = x**4 - 4*x**2 + 3




Аналогично найдём $G1 = G \cap k[y]$. Для этого нам нужен лексикографический порядок $x > y$.

In [22]:
F = [x**2 + 2*y**2 - 3, x**2 + x*y + y**2 - 3]           # Идеал I
g = groebner(F, x, y, order='lex')                       # Базис Грёбнера G
print('\nБазис Грёбнера относительно x > y:')
for i in range(len(g.exprs)):
    print('g', i+1, ' = ', g.exprs[i], sep='')

polys = g.exprs
G1 = []
index = 1
I0 = []
for pol in polys:
    f = Poly(pol, y)                                     # Представим каждое выражение как полином от переменной Y
    if f.free_symbols_in_domain != {x}:                  # Если помимо переменной Y в полиноме НЕТ свободной переменной X,
        G1.append((f.as_expr(), index))                  # то этот полином содержится в G1
    else:
        I0.append((Poly(f, x), index))                   # Запомним отдельно полиномы, которые не содержатся в G1.
    index += 1
print('\nБазис Грёбнера 1-ого исключающего идеала:')     
for pol in G1:
    print('g', pol[1], ' = ', pol[0], sep='')
print('\n')


Базис Грёбнера относительно x > y:
g1 = x**2 + 2*y**2 - 3
g2 = x*y - y**2
g3 = y**3 - y

Базис Грёбнера 1-ого исключающего идеала:
g3 = y**3 - y




Заметим, что многочлен $g_3$ зависит только от одной переменной $y$. Мы сделали **шаг исключения**, избавившись от переменной $x$. Решив это уравнение, получим возможные значения $y$ для решения всей системы - это будет **частичным решением**.

По **теореме о продолжении** следующий шаг - шаг продолжения - может не выполниться (т.е. частичное решение не верно для всей системы), если коэффициенты при старших мономах одновременно равны нулю. Сделаем проверку на это для остальных полиномов, не содержащихся в $G_1$

In [23]:
for f in G1:
    pol = Poly(f[0], y)
    roots = pol.all_roots()                               # Нашли корни методом .all_roots()
    print('корни:', roots)
    
for root in roots:
    for pol in I0:
        f = Poly(pol[0], x)
        if root not in solve(LC(f), y):                   # Проверка, обращает ли в ноль частичное решение старшие мономы
            break                                         # полиномов из I0
    else:
        roots.remove(root)
        
print('корни после проверки:', roots, '\n')

корни: [-1, 0, 1]
корни после проверки: [-1, 0, 1] 



Теперь сделаем **шаг продолжения**. Найдём все решения системы и сразу определим, какие из решений рациональны.

In [24]:
rationals = []
print('Все решения системы:')
for root in roots:
    new_polys = []
    for pol in I0:
        new_polys.append(pol[0].subs(y, root))            # Подставим значения частичных решений методом .subs(y,root)
    solution = solve_poly_system(new_polys, x)            # и решим систему уравнений от одной переменной X
    for sol in solution:
        if sol[0].is_rational and root.is_rational:
            rationals.append((sol[0], root))
        else:
            if sol[0].is_real and root.is_real:
                real = True
            else:
                comp = True
        print('(', sol[0], ', ', root, ')', sep='')

print('\nРациональные решения системы:')
print(rationals)
print('Наименьшее поле k такое, что все решения принадлежат k^2, - это', 'R' if real else 'C')


Все решения системы:
(-1, -1)
(-sqrt(3), 0)
(sqrt(3), 0)
(1, 1)

Рациональные решения системы:
[(-1, -1), (1, 1)]
Наименьшее поле k такое, что все решения принадлежат k^2, - это R
